# Trabalho de Inteligência Artificial - 2019/2

Gustavo Eugênio de Souza Moraes RA: 620238<br>
Gabriel Givigi Oliveira RA: 760924

### Bibliotecas utilizadas e variáveis globais

In [1]:
from pandas import DataFrame, read_csv # for reading tsv files and structuring
import math # for mathematical functions
import numpy as np # for vector stuff
import matplotlib.pyplot as plt # for plotting graphs
from random import randint # for generating random colorse
#from sklearn.metrics.cluster import adjusted_rand_score # for rand index measure

dataset1 = read_csv("datasets/c2ds1-2sp.txt", sep="\t") # Lê o arquivo de dados
dataset2 = read_csv("datasets/c2ds3-2g.txt", sep="\t") # Lê o arquivo de dados
dataset3 = read_csv("datasets/monkey.txt", sep="\t") # Lê o arquivo de dados

# funcção de distância euclidiana
def myeuclidean(p, q):
    return math.sqrt((p['d1']-q['d1'])**2+(p['d2']-q['d2'])**2);

### Algoritmo K-Médias

Aqui temos nossa implementação do algoritmo K-Médias, feita sem a necessidade de consulta apenas seguindo as etapas do algoritmo demonstrads em aula. O código se divide em 4 etapas, sendo elas:
1. <b>INICIALIZAÇÃO</b><br>
    &ensp; A primeira etapa consiste em definir os k-centróides iniciais necessários para o funcionamento do algortimo. Nossa solução foi definir como centróides iniciais os k primeiros valores do <em>dataframe</em> formado. 
2. <b>(RE)CALCULAR CENTRÓIDE</b><br>
    &ensp; O cálculo do centróide é feito de maneira simples, nós montamos uma <em>view</em> do <em>dataframe</em> filtrando pela coluna <em>"cluster_id"</em>. Em seguida com a função mean() tiramos a média aritmética de cada grupo marcado pelo ID do cluster, e atualizamos seu centróide.
3. <b>TIRAR DISTANCIA</b><br>
    &ensp; Para tirar as distancias de cada amostra até os centróides, selecionamos todas as <em>rows</em> do <em>dataframe</em> e, uma a uma, utilizamos a função myeuclidean() para realizar o cálculo.
4. <b>DEFINIFIR GRUPOS</b><br>
    &ensp; Tiradas as distâncias, com a combinação de funções index() e min(), determinamos qual o centróide mais próximo, para com isso, rotular corretamente o cluster adequado para a amostra.

In [2]:
def mykmeans(k, data, n_iterations):
    for i in range(0, k): # initialize centroids with k first rows
        data.loc[i, 'cluster_id'] = int(i)
    for j in range(0, n_iterations): # run n_iteration times
        for i in range(0, k): # (re)calc centroids
            this = data[data.cluster_id == i] # capture all samples from centroid i
            centroid.loc[i] = {'d1':this.d1.mean(), 'd2':this.d2.mean(), 'cluster_id':i} # atualizing centroid i
        for index_data, row_data in data.iterrows(): # for each sample
            calculated_distance = []
            for index_centroid, row_centroid in centroid.iterrows(): # checks each sample and takes the distances
                calculated_distance.append(myeuclidean(row_data, row_centroid))
            row_data.loc['cluster_id'] = calculated_distance.index(min(calculated_distance)) # saves the shortest distance
            data.iloc[index_data] = row_data # sends it back to the dataframe
            
    return data;

### Algoritmo Single-Link

Para a implementação da função de agrupamento Single-Link, o grupo recorreu a material de apoio disponibilizado na internet, checando sua implementação em diversos sites e até observando algoritmos otimizados de grandes bibliotecas confiáveis, como a SciPy. O grupo também informa que, mesmo tendo todo esse material disponível, para fins didáticos, tentou seguir com sua própria implementação adaptada desses materiais. Sendo assim, vamos à descrição do algoritmo.<br>
1. <b>INICIALIZAÇÃO</b><br>
    &emsp; Para a inicialização do algoritmo, declaramos cada amostra disponível no <em>dataframe</em> como um cluster, e jogamos todos na lista de clusters.
2. <b>SIMULAR MATRIZ DE DISTÂNCIAS</b><br>
    &emsp; O próximo passo, repetindo para cada cluster até obtivermos o número desejado k de clusters final, é determinar o par de amostras, de clusters diferentes, que têm a menor distância entre si. Isso é feito comparando todas as amostras numa matriz de distâncias de tamanho N, sendo N o número de amostras total. <br>É trivial dizer que a complexidade desse algoritmo é elevada nesta implementação, o que acabou trazendo dificuldades para o grupo em sua execução mais à frente no relatório.
3. <b>ATRIBUIÇÕES E REMOÇÕES</b><br>
    &emsp; Determinada a matriz de distância, pegamos o par de amostrar com menor distância entre si, se unimos seus clusters, até termos o numero total k desejado. 

In [3]:
def mysinglelink(data, kmin, kmax):
    # initialization of clusters at first (every point is a cluster)
    clusters=[]
    ret_clusters=[]
    for index, row in data.iterrows():
        clusters.append([row]) # initialize with a cluster for every sample
    while (len(clusters) > kmin): # check for n_cluster == k
        menor_dist = clust_increment = clust_merge = math.inf # set as infinite
        for cluster_id, cluster in enumerate(clusters[:len(clusters)]): # cicle though every cluster group 
            for sample_id, sample in enumerate(cluster): # cicle through every data in cluster group 1
                for cluster_merge_id, cluster_merge in enumerate(clusters[(cluster_id+1):]): # cicle through every cluster group besides the first one
                    for another_sample_id, another_sample in enumerate(cluster_merge): # cicle through every data in cluster group 1
                        if myeuclidean(sample, another_sample) < menor_dist: # check for distance
                            menor_dist = myeuclidean(sample, another_sample) # saves the distance if less than now
                            clust_increment = cluster_id # saves the cluster to extend
                            clust_merge = cluster_merge_id+cluster_id+1 # saves the cluster for removal
        clusters[clust_increment].extend(clusters[clust_merge]) # merge cluster's copy
        clusters.pop(clust_merge) # removes merged cluster's original copy
        if len(clusters) <= kmax: # saves multiple copies from kmax to kmin
            ret_clusters.append(clusters.copy()) 
    return(ret_clusters); # returns formed clusters

### Algoritmo Average-Link

Para a implementação do algoritmo Average-Link, utilizamos da mesma estrutura feita para o Single-Link, mudando apenas suas métricas.<br>
1. <b>INICIALIZAÇÃO</b><br>
    &emsp; Para a inicialização do algoritmo, declaramos cada amostra disponível no <em>dataframe</em> como um cluster, e jogamos todos na lista de clusters.
2. <b>TIRAR A DISTÃNCIA MÉDIA ENTRE CLUSTERS</b><br>
    &emsp; Agora, diferentemente da implementação do Single-Link, nós iteramos para cada combinação de par de centróides, e novamente para todas as possibilidades de combinações de amostras entre esses mesmos clusters, salvando a soma total de distâncias em uma variável e o número total de distâncias calculadas. Com esses dados, podemos tirar a média de distância entre 2 clusters, para então, calcularmos qual a menor média de distâncias na iteração, e unir os clusteres correspondentes à essa união. <br>Novamente, é trivial dizer que a complexidade desse algoritmo é elevada nesta implementação, o que acabou trazendo dificuldades para o grupo em sua execução mais à frente no relatório.
3. <b>ATRIBUIÇÕES E REMOÇÕES</b><br>
    &emsp; Determinada a matriz de distância, pegamos o par de amostrar com menor distância entre si, se unimos seus clusters, até termos o numero total k desejado. 

In [4]:
def myaveragelink(data, kmin, kmax):
    # initialization of clusters at first (every point is a cluster)
    clusters=[]
    ret_clusters=[]
    for index, row in data.iterrows():
        clusters.append([row]) # initialize with a cluster for every sample
    while (len(clusters) > kmin): # check for n_cluster == k
        menor_dist = clust_increment = clust_merge = math.inf # set as infinite
        for cluster_id, cluster in enumerate(clusters[:len(clusters)]): # cicle though every cluster group 
            for cluster_merge_id, cluster_merge in enumerate(clusters[(cluster_id+1):]): # cicle through every cluster group besides the first one
                total_distance = 0 # reset sum of distances
                n_distances = 0 # reset num of distances
                for sample_id, sample in enumerate(cluster): # cicle through every data in cluster group 1
                    for another_sample_id, another_sample in enumerate(cluster_merge): # cicle through every data in cluster group 1
                        total_distance += myeuclidean(sample, another_sample) # increments total distance
                        n_distances += 1 # increments num of distances
                avg_dist = total_distance / n_distances # gets the average distance of 2 clusters
                if avg_dist < menor_dist: # checks for new menor_dist
                    menor_dist = avg_dist # saves the distance if less than now
                    clust_increment = cluster_id # saves the cluster to extend
                    clust_merge = cluster_merge_id+cluster_id+1 # saves the cluster for removal
        clusters[clust_increment].extend(clusters[clust_merge]) # merge cluster's copy
        clusters.pop(clust_merge) # removes merged cluster's original copy
        if len(clusters) <= kmax: # saves multiple copies from kmax to kmin
            ret_clusters.append(clusters.copy()) 
    return(ret_clusters); # returns formed clusters

# Rodando os algoritmos

### Primeiramente, o K-Médias rodando o primeiro dataset (c2ds1-2sp.txt):

In [5]:
# escolha aqui o valor de k e o numero de iterações
k_list = [2, 3, 4, 5]
n_iterations = 10

kmeans1 = []
for k in k_list:
    manipulated_data = dataset1.copy() # setting up manipulated dataset for use
    manipulated_data['cluster_id'] = None # adding a column for cluster
    centroid = DataFrame(columns = ['d1', 'd2', 'cluster_id']) # setting up clusters dataframe
    kmeans1.append(mykmeans(k, manipulated_data, n_iterations)) # salva o resultado

    kmeans1[-1].plot.scatter('d1', 'd2', c='cluster_id', colormap='gist_rainbow')
    plt.suptitle("K-Means for dataset 1 (c2ds1-2sp) with k = %d" % k)
    plt.show()

TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [ ]:
filepath = "/home/thaina/Downloads/kmeans"
i = 2
for cluster in kmeans1:
    view = cluster.copy()
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i += 1

### Análise dos resultados, K-Médias(c2ds1-2sp):
&emsp; Como podemos notar nas imagens acima, um algoritimo de agrupamento baseado em centro não é eficaz para separar adequadamente amostras como as do dataset1, dado que as amostras são bem decentralizadas e sequênciais.<br>
&emsp; Podemos concluir que o resultado obtido foi longe do ideal, resultando apenas em uma "setorização" da imagem, dividindo-a em partes praticamente siméticas, porém sem nenhum valor embutido para análise.

### K-Médias rodando o segundo dataset (c2ds3-2g.txt):

In [ ]:
# escolha aqui o valor de k e o numero de iterações
k_list = [2, 3, 4, 5]
n_iterations = 10

kmeans2 = []
for k in k_list:
    manipulated_data = dataset2.copy() # setting up manipulated dataset for use
    manipulated_data['cluster_id'] = None # adding a column for cluster
    centroid = DataFrame(columns = ['d1', 'd2', 'cluster_id']) # setting up clusters dataframe
    kmeans2.append(mykmeans(k, manipulated_data, n_iterations)) # salva o resultado

    kmeans2[-1].plot.scatter('d1', 'd2', c='cluster_id', colormap='gist_rainbow')
    plt.suptitle("Dataset 2 (c2ds3-2g) with k = %d" % k)
    plt.show()

In [ ]:
filepath = "/home/thaina/Downloads/kmeans/"
i = 2
for cluster in kmeans2:
    view = cluster.copy()
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i += 1

### Análise dos resultados, K-Médias(c2ds3-2g):
&emsp; Para o dataset utilizado neste caso, agora sim temos um ótimo candidado para utilização de algoritmos de agrupamento baseados em centros. Podemos notar que o algoritmos separou com perfeição os dados utilizando k = 2, detectando os dois grandes centros de aglomeração de amostras e separando-os minuciosamente; Além de fazer uma boa separação para k = 3, detectando os dois mesmos grandes centros, mas separando também, uma "área de colisão", região onde os dados, apesar de estarem divididos entre os 2 grandes centros, têm um grau de semelhança considerável.<br>
&emsp; Para k's maiores que 3, os agrupamentos parecem começar a ficar distorcidos e carecer de informação.

### K-Médias rodando o terceiro e último dataset (monkey.txt):

In [ ]:
# escolha aqui o valor de k e o numero de iterações
k_list = [5, 6, 7, 8,  9, 10, 11, 12]
n_iterations = 10

kmeans3 = []
for k in k_list:
    manipulated_data = dataset3.copy() # setting up manipulated dataset for use
    manipulated_data['cluster_id'] = None # adding a column for cluster
    centroid = DataFrame(columns = ['d1', 'd2', 'cluster_id']) # setting up clusters dataframe
    kmeans3.append(mykmeans(k, manipulated_data, n_iterations)) # salva o resultado
    
    kmeans3[-1].plot.scatter('d1', 'd2', c='cluster_id', colormap='gist_rainbow')
    plt.suptitle("Dataset 3 (monkey) with k = %d" % k)
    plt.show()

In [ ]:
filepath = "/home/thaina/Downloads/kmeans/d3/"
i = 5
for cluster in kmeans3:
    view = cluster.copy()
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i += 1

### Análise dos resultados, K-Médias(monkey):
&emsp; Vamos agora para a análise dos agrupamentos formados no dataset 3(monkey).<br>
&emsp; Para os agrupamentos formados com k entre 5 e 6, notamos que não houve efetividade do algoritmo para fazer a clusterização. É notável que, como no caso do dataset 1, o algoritmo apenas "setorizou" a imagem, dividindo-a com "linhas imaginárias".<br>
&emsp; No restante dos agrupamentos, pode-se perceber uma sutil melhora na clusterização, onde o algoritimo demonstra a capacidade de reconhecer parcialmente as divisões menos espaçadas; Sendo elas as "orelhas" do macaco, seus "olhos" e sobrancelhas. Traços decentralizados como o contorno da "cabeça" e "boca" tiveram pouca efetivade nos agrupamentos gerados.

### Agora, o Single-Link rodando o primeiro dataset (c2ds1-2sp.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 2
kmax = 5

# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 1
view = dataset1
#clusters_s1 = mysinglelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters_s1):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
    plt.show()

In [ ]:
filepath = "/home/debora/Documents/IA/Trab/singlelink/d1/"
i = 5

view = DataFrame(columns = ['sample_label','d1', 'd2', 'cluster_id'])
test = clusters_s1.copy()

single1 = []
for k_clusters_index, k_clusters in enumerate(test):
    listing = []
    for cluster_index, cluster in enumerate(k_clusters):
        for sample in cluster:
            sample['cluster_id'] = cluster_index
        listing += cluster
    view = DataFrame(listing, columns = ['sample_label','d1', 'd2', 'cluster_id'])
    view.sort_index(inplace=True)
    view.drop_duplicates(keep='first', inplace=True)
    single1.append([i, view])
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i -= 1

### Análise dos resultados, Single-Link(c2ds1-2sp):
&emsp; Após logo o primeiro exemplo deste <em>dataset</em>, com k = 2, podemos perceber a efetividade do algoritmo Single-Link para dados ditos "encadeados", separando perfeitamente as duas "linhas" formadas pelos dados.<br>
&emsp; Podemos também perceber sua constância na geração dos clusters para k's maiores que 2, onde, geralmente, os clusters que não os dois encadeamentos principais, normalmente constam de 1 dado apenas (o dado que têm a maior distância para um próximo).

### Single-Link rodando o segundo dataset (c2ds3-2g.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 2
kmax = 5

# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 1
view = dataset2
#clusters_s2 = mysinglelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters_s2):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
    plt.show()

In [ ]:
filepath = "/home/debora/Documents/IA/Trab/singlelink/d2/"
i = 5

view = DataFrame(columns = ['sample_label','d1', 'd2', 'cluster_id'])
test = clusters_s2.copy()

single2 = []
for k_clusters_index, k_clusters in enumerate(test):
    listing = []
    for cluster_index, cluster in enumerate(k_clusters):
        for sample in cluster:
            sample['cluster_id'] = cluster_index
        listing += cluster
    view = DataFrame(listing, columns = ['sample_label','d1', 'd2', 'cluster_id'])
    view.sort_index(inplace=True)
    view.drop_duplicates(keep='first', inplace=True)
    single2.append([i, view])
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i -= 1

### Análise dos resultados, Single-Link(c2ds3-2g):
&emsp; Com este exemplo, podemos perceber agora a inefetividade do algoritmo Single-Link para dados centralizados, que demonstra incapacidade de agrupar praticamente todos os casos de maneira eficaz.<br>
&emsp; Como no dataset anterior, normalmente dados considerados "outlyers" pelo algorimto são tratados como clusters unitários, sendo compostos pelos dados mais afastados de um vizinho qualquer.

### Single-Link rodando o terceiro e último dataset (monkey.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 5
kmax = 12

single1 = []
# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 1
view = dataset3
clusters = mysinglelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
    plt.show()

### Por fim, o Average-Link rodando o primeiro dataset (c2ds1-2sp.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 2
kmax = 5

# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 1
view = dataset1
clusters_a1 = myaveragelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters_a1):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
    plt.show()

In [ ]:
filepath = "/home/debora/Documents/IA/Trab/averagelink/d1/"
i = 5

view = DataFrame(columns = ['sample_label','d1', 'd2', 'cluster_id'])
test = clusters_a1.copy()

average1 = []
for k_clusters_index, k_clusters in enumerate(test):
    listing = []
    for cluster_index, cluster in enumerate(k_clusters):
        for sample in cluster:
            sample['cluster_id'] = cluster_index
        listing += cluster
    view = DataFrame(listing, columns = ['sample_label','d1', 'd2', 'cluster_id'])
    view.sort_index(inplace=True)
    view.drop_duplicates(keep='first', inplace=True) 
    average1.append([i, view])
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i -= 1

### Average-Link rodando o segundo dataset (c2ds3-2g.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 2
kmax = 5

average2 = []
# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 2
view = dataset2
clusters_a2 = myaveragelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters_a2):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
    plt.show()

In [ ]:
filepath = "/home/debora/Documents/IA/Trab/averagelink/d2/"
i = 5

view = DataFrame(columns = ['sample_label','d1', 'd2', 'cluster_id'])
test = clusters_a2.copy()

average2 = []
for k_clusters_index, k_clusters in enumerate(test):
    listing = []
    for cluster_index, cluster in enumerate(k_clusters):
        for sample in cluster:
            sample['cluster_id'] = cluster_index
        listing += cluster
    view = DataFrame(listing, columns = ['sample_label','d1', 'd2', 'cluster_id'])
    view.sort_index(inplace=True)
    view.drop_duplicates(keep='first', inplace=True) 
    average2.append([i, view])
    view.to_csv(path_or_buf=filepath+str(i)+".clu", header=False , columns=['sample_label', 'cluster_id'], sep="\t", index=False)
    i -= 1

### Average-Link rodando o terceiro e último dataset (monkey.txt):

In [ ]:
# escolha o valor de k aqui
kmin = 5
kmax = 12

average3 = []
# color generating
colors = []
for i in range(kmax):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# plot generating for dataset 3
view = dataset3
clusters = myaveragelink(view, kmin, kmax)
for k_clusters_index, k_clusters in enumerate(clusters):
    for cluster_index, cluster in enumerate(k_clusters):
        for sample_index, sample in enumerate(cluster):
            plt.plot([sample['d1']], [sample['d2']], marker='o', markersize=3, color=colors[cluster_index])
plt.show()

### Aplicando o Índice Rand Ajustado

In [ ]:
expected1 = read_csv("datasets/c2ds1-2spReal.clu", sep="\t", header=None) # Lê o arquivo de dados
expected2 = read_csv("datasets/c2ds3-2gReal.clu", sep="\t", header=None) # Lê o arquivo de dados
expected3 = read_csv("datasets/monkeyReal1.clu", sep="\t", header=None) # Lê o arquivo de dados

test = read_csv("singlelink/d3/12.clu", sep="\t", header=None)

display(adjusted_rand_score(test[1], expected3[1]))